In [1]:
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, Flatten , Dense, Dropout, Lambda
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping

from load_data import split_data, generator

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size=32
augment_data=True
bias=0.8
no_epochs=50

In [3]:
def create_model(input_shape=(66, 200, 3)):
    
    input_layer = Input(shape=input_shape)
    x = Lambda(lambda z: z/127.5 - 1.)(input_layer)
    x = Conv2D(filters=24, kernel_size=(5,5), strides=(2,2), activation='elu')(input_layer)
    x = Conv2D(filters=36, kernel_size=(5,5), strides=(2,2), activation='elu')(x)
    x = Conv2D(filters=48, kernel_size=(5,5), strides=(2,2), activation='elu')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='elu')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), activation='elu')(x)
    x = Dropout(0.5)(x)

    x = Flatten()(x)
    
    x = Dense(units=100, activation='elu')(x)
    x = Dense(units=50, activation='elu')(x)
    x = Dense(units=10, activation='elu')(x)
    output_layer = Dense(units=1)(x)
    
    model = Model(input=input_layer, output=output_layer)
    
    print(model.summary())
    return model

In [4]:
model = create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 66, 200, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 18, 64)         0         
__________

C:\Users\windo\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [5]:
train_data, validation_data = split_data()

In [6]:
model.compile(optimizer='adam', loss='mse')

In [7]:
checkpointer = ModelCheckpoint('checkpoints/weights.{epoch:02d}-{val_loss:.3f}.hdf5')
logger = CSVLogger(filename='logs/history.csv')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=10)

with open('logs/model.json', 'w') as f:
    f.write(model.to_json())

model.fit_generator(generator=generator(train_data, batch_size=batch_size, augment_data=augment_data, bias=bias),
                     steps_per_epoch=len(train_data)/batch_size,
                     epochs=no_epochs,
                     validation_data=generator(validation_data, augment_data=False, bias=1.0),
                     validation_steps=len(validation_data)/batch_size,
                     callbacks=[checkpointer, logger, early_stop])

Epoch 1/50
113/112 [==============================] - 17s 150ms/step - loss: 0.2774 - val_loss: 0.0537
Epoch 2/50
113/112 [==============================] - 14s 121ms/step - loss: 0.0626 - val_loss: 0.0551
Epoch 3/50
113/112 [==============================] - 14s 120ms/step - loss: 0.0499 - val_loss: 0.0312
Epoch 4/50
113/112 [==============================] - 15s 134ms/step - loss: 0.0387 - val_loss: 0.0251
Epoch 5/50
113/112 [==============================] - 15s 133ms/step - loss: 0.0343 - val_loss: 0.0243
Epoch 6/50
113/112 [==============================] - 14s 124ms/step - loss: 0.0343 - val_loss: 0.0258
Epoch 7/50
113/112 [==============================] - 14s 123ms/step - loss: 0.0287 - val_loss: 0.0238
Epoch 8/50
113/112 [==============================] - 15s 129ms/step - loss: 0.0300 - val_loss: 0.0219
Epoch 9/50
113/112 [==============================] - 14s 121ms/step - loss: 0.0284 - val_loss: 0.0212
Epoch 10/50
113/112 [==============================] - 13s 119ms/step - l